In [1]:
# Verify we're in the correct working directory
import os
os.getcwd()   

'/Users/seraphinashi/Desktop/l3c_ctml'

In [2]:
from src.STEP1_feature import a_cohort as a
from src.STEP1_feature import b_medication as b
import os
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType

In [3]:
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("l3c_ctml") \
        .getOrCreate()

# Read CSV file into table
data_path = "synthetic_data/training"

schema = StructType([
    StructField('table', StringType(), True),
    StructField('field', StringType(), True),
    StructField('type', StringType(), True)
])

all_fields = spark.createDataFrame([], schema)
    
for path, subdirs, files in os.walk(data_path):
    for name in files:
        if "csv" in name and name[0]!=".":
            # table_fields = [(table_name, f.name, str(f.dataType)) for f in datafile.schema.fields]
            full_file = (os.path.join(path, name))
            df = spark.read.csv(os.path.join(path, name), header=True, inferSchema=True)
            var_name = name[:-4]
            print(full_file)
            globals()[var_name] = df

concept = spark.read.csv("synthetic_data/concept.csv", header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 15:19:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
synthetic_data/training/observation_period.csv


synthetic_data/training/drug_exposure.csv
synthetic_data/training/death.csv


synthetic_data/training/measurement.csv


synthetic_data/training/condition_occurrence.csv


synthetic_data/training/microvisits_to_macrovisits.csv


synthetic_data/training/visit_occurrence.csv
synthetic_data/training/long_covid_silver_standard.csv
synthetic_data/training/person.csv


synthetic_data/training/observation.csv


synthetic_data/training/procedure_occurrence.csv


In [4]:
# 1_cohort_and_features

covid_pasc_index_dates = a.sql_statement_01(long_covid_silver_standard)
cohort = a.sql_statement_00(covid_pasc_index_dates, person)
long_covid_patients = a.sql_statement_08(covid_pasc_index_dates)

hosp_cases = a.sql_statement_04(cohort, condition_occurrence, microvisits_to_macrovisits)

hosp_and_non = a.sql_statement_06(cohort, hosp_cases)

Feature_Table_Builder_v0 = a.sql_statement_03(covid_pasc_index_dates, hosp_and_non, microvisits_to_macrovisits)
# ICU_visits = a.sql_statement_05(microvisits_to_macrovisits)
# inpatient_visits = spark.sql(sql_statement_07())

In [5]:
# 2_med_feature_table
DrugConcepts = b.sql_statement_00(concept)

In [14]:
from pyspark.sql.functions import col

def exp(Feature_table_builder, drug_exposure):
    d = drug_exposure.alias("d")
    f = Feature_table_builder.alias("f")

    result = d.join(f, col("d.person_id") == col("f.person_id"), "inner") \
            .select("d.*")
    return(result)

Drugs_for_These_Patients = exp(Feature_Table_Builder_v0, drug_exposure)

AnalysisException: cannot resolve 'drug_exposure.*' given input columns 'person_id, drug_exposure_start_date, refills, drug_source_value, drug_exposure_end_date, route_concept_id, quantity, lot_number, days_supply, sig, drug_type_concept_id, drug_source_concept_id, provider_id, route_source_value, drug_exposure_id, dose_unit_source_value, drug_exposure_start_datetime, drug_exposure_end_datetime, visit_detail_id, verbatim_end_date, stop_reason, drug_concept_id, visit_occurrence_id, person_id, patient_group, apprx_age, sex, race, ethn, min_covid_dt_2, tot_long_data_days, pre_pre_window_start_dt, pre_window_start_dt, pre_window_end_dt, post_window_start_dt, post_window_end_dt, post_visits_count, tot_post_days, tot_covid_days, op_post_visit_ratio'

In [ ]:


    drugRollUp = b.sql_statement_04()

    covid_drugs = b.sql_statement_02()
    pre_pre_drugs = b.sql_statement_10()
    pre_drugs = b.sql_statement_07()
    post_drugs = b.sql_statement_05()

    covidtbl = b.sql_statement_03()
    prepretbl = b.sql_statement_11()
    pretbl = b.sql_statement_12()
    posttbl = b.sql_statement_06()

    pre_post_med_count = b.sql_statement_08()

    pre_post_med_count_clean = b.sql_statement_09()